# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [2]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [86]:
import numpy as np
import random

def variable_depth_local_search(graph, constant, depth):

    def get_weight1(p1, p2):
        # Ищем вес текущего разбиения.
        # Никакой дополнительной информации нет.
        nonlocal d1
        nonlocal d2
        d1 = np.zeros(len(graph[0]) + 2)
        d2 = np.zeros(len(graph[0]) + 2)
        weight = 0
        for (v1,v2) in graph[1]: # Выгоднее перебирать все рёбра, чем все пары вершин.
            if( v1 in p1 and v2 in p2)or(v1 in p2 and v2 in p1):
                weight += 1
                if(v1 in p1):
                    d2[v1] += 1 # Учитываем каждое ребро для подсчёта связей.
                    d1[v2] += 1
                else:
                    d1[v1] += 1
                    d2[v2] += 1
            else:
                if(v1 in p1):
                    d1[v1] += 1
                    d1[v2] += 1
                else:
                    d2[v1] += 1
                    d2[v2] += 1
                    
        return weight//2 # Все рёбра посчитаны дважды.
    
    
    def get_new_weight(v1, v2, weight):
        # Ищем вес текущего разбиения.
        # При этом известен вес старого разбиения и пара вершин, которые поменялись местами.
        nonlocal d1
        nonlocal d2
        ans = weight - (d1[v2] + d2[v1] - d2[v2] - d1[v1])//2;
        return ans
    
    def try_changing(p1, p2):
        # Попытка улучшить текущий результат.
        # Возвращаем вес оптимального разбиения в 1-окрестности.
        nonlocal d1
        nonlocal d2
        nonlocal passive_part1
        nonlocal passive_part2
        box1 = set() # Пара вершин, которую нужно перекинуть.
        box2 = set() #
        local_best_weight = get_weight1(p1 | passive_part1, p2 | passive_part2) # Лучший вес.
        base_weight = local_best_weight # Вес исходной конфигурации.
        fail = 0 # Максимальное количество неудачных попыток подряд.
        while fail < constant:
            first = random.randint(1, len(graph[0]))
            while(first not in p1):
                first = random.randint(1, len(graph[0]))
            second = random.randint(1, len(graph[0]))
            while(second not in p2):
                second = random.randint(1, len(graph[0]))
            new_weight = get_new_weight(first, second, base_weight)
            if(local_best_weight > new_weight): # Результат улучшился.
                box1 = {first}
                box2 = {second}
                local_best_weight = new_weight
                fail = 0
            else:
                fail += 1
                
        return (p1 - box1) | box2, (p2 - box2) | box1, local_best_weight
    
    def choose_passive(c_part1, c_part2):
        # "Молния".
        # Ухудшаем текущее разбиение, пытаясь выбраться из локального оптимума.
        nonlocal d1
        nonlocal d2
        nonlocal passive_part1
        nonlocal passive_part2
        
        passive1 = random.randint(1, len(graph[0]))
        while not (passive1 in cur_part1):
            passive1 = random.randint(1, len(graph[0]))
            
        passive2 = random.randint(1, len(graph[0]))
        while not passive2  in cur_part2:
            passive2 = random.randint(1, len(graph[0]))
            
        passive_part1 = {passive1} # Запрещаем вершинам уходить в другую долю.
        passive_part2 = {passive2}
        c_part1 = c_part1 - passive_part1 # Новые доли разбиения.
        c_part2 = c_part2 - passive_part2
        
        
    opt_part1 =  set(range(1, len(graph[0]) // 2 + 1)) # Самое оптимальное из найденных.
    opt_part2 =  set(range(len(graph[0]) // 2 + 1, len(graph[0])))
    opt_passive_part1 = set() # Вершины, которые нельзя перемещать.
    opt_passive_part2  = set()
    
    cur_part1= opt_part1.copy() # Текущее разбиение.
    cur_part2 = opt_part2.copy()
    passive_part1 = set() # Вершины, которые нельзя перемещать.
    passive_part2 = set()
    fails = 0
    d1 = np.zeros(len(graph[0])+1) # Оптимизация: количество ребер из вершины x в вершины первой доли.
    d2 = np.zeros(len(graph[0])+1) # Аналогично для второй доли.
    #starting_point = set(range(1, len(graph[0]) // 2 + 1))

    cur_weight = get_weight1(opt_part1, opt_part2) # Вес текущего разбиения.
    result = cur_weight # Наилучший результат поиска в окрестности.
    best_weight = cur_weight # Вес оптимального разбиения.
    while fails < depth: # Максимальный размер "молнии" без изменений.
        # Повторяем, пока результат улучшается. 
        cur_part1, cur_part2, result = try_changing(cur_part1, cur_part2)
        cur_weight = result
        if(cur_weight < best_weight): # Нашли новый оптимум.
            result = cur_weight
            best_weight = cur_weight
            opt_part1 = cur_part1
            opt_part2 = cur_part2
            opt_passive_part1 = passive_part1
            opt_passive_part2 = passive_part2
            passive_part1 = set() # Разрешаем всем вершинам пермещаться
            passive_part2 = set()
            fails = 0 
        else:
            fails += 1 # Не повезло.
            cur_part1 = opt_part1
            cur_part2 = opt_part2
            choose_passive(cur_part1, cur_part2) # Выбираем новые вершины для "молнии".
    return opt_part1 | opt_passive_part1

In [89]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph','cti.graph', 't60k.graph', 'm14b.graph']
    constants = [3000, 2000, 2000, 200]
    depths = [30, 300, 300, 5]
    for filename, constant, depth in zip(filenames, constants, depths):
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance, constant, depth))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [90]:
run_all()

Solving instance add20.graph… done in 2.9e+01 seconds with quality 12724
Solving instance cti.graph… done in 3.9e+01 seconds with quality 94052
Solving instance t60k.graph… done in 8.6e+01 seconds with quality 178236
Solving instance m14b.graph…

KeyboardInterrupt: 

## Выводы
(Здесь опишите свои наблюдения и подобранные параметры для каждого из четырёх входных графов.)

1. Поскольку полный перебор 1-окрестностей неприемлем по времени для больших тестов, возможен перебор только некоторой части этой окрестности.

2. Из-за ограниченности времени можно не запускаться рекурсивно после каждой новой "молнии", а увеличивать размер множества вершин, которых нельзя перемещать, добавляя к ним новые, то есть удлинять "молнию" без разветвлений. Так мы имеем возможность уйти дальше от локального оптимума.

3. Для графов из 1 и 4 тестов первоначальное разбиение заметно улучшается. Для 1 теста есть возможность выбирать наугад больше пар вершин для обменов, поскольку граф невелик.

4. Для графов из 2 и 3 тестов результат почти не улучшается. Для них даже большой размер "молнии" не приводит к существенным изменениям. Значит, у этих графов первоначальное разбиение даёт очень "глубокий" оптимум.